In [1]:
# ================================
# CPTAC-LUAD (LinkedOmics) -> CSVs
# Creates:
#   data/transcriptomics_processed.csv
#   data/proteomics_processed.csv
# ================================

!pip -q install pandas numpy

import os, requests
import pandas as pd

os.makedirs("data", exist_ok=True)

# --- LinkedOmics CPTAC-LUAD direct files (from the portal page) ---
# RNAseq (Tumor / Normal)
RNA_TUMOR_URL  = "https://linkedomics.org/data_download/CPTAC-LUAD/HS_CPTAC_LUAD_rnaseq_uq_rpkm_log2_NArm_TUMOR.cct"
RNA_NORMAL_URL = "https://linkedomics.org/data_download/CPTAC-LUAD/HS_CPTAC_LUAD_rnaseq_uq_rpkm_log2_NArm_NORMAL.cct"

# Proteome (Tumor / Normal) - gene-level TMT log2 ratio
PROT_TUMOR_URL  = "https://linkedomics.org/data_download/CPTAC-LUAD/HS_CPTAC_LUAD_proteome_ratio_NArm_TUMOR.cct"
PROT_NORMAL_URL = "https://linkedomics.org/data_download/CPTAC-LUAD/HS_CPTAC_LUAD_proteome_ratio_NArm_NORMAL.cct"

def download(url: str, out_path: str):
    r = requests.get(url, stream=True, timeout=120)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
    print("Downloaded:", out_path)

def read_cct(path: str) -> pd.DataFrame:
    """
    LinkedOmics *.cct continuous matrix:
    typically tab-delimited with features in rows and samples in columns.
    We'll read it as: rows=features, cols=samples, then transpose later.
    """
    df = pd.read_csv(path, sep="\t", header=0)
    # First column is feature ID (gene symbol); set as index
    df = df.set_index(df.columns[0])
    return df

# 1) Download the four matrices
download(RNA_TUMOR_URL,  "data/rna_tumor.cct")
download(RNA_NORMAL_URL, "data/rna_normal.cct")
download(PROT_TUMOR_URL, "data/prot_tumor.cct")
download(PROT_NORMAL_URL,"data/prot_normal.cct")

# 2) Read matrices (features x samples)
rna_tumor  = read_cct("data/rna_tumor.cct")
rna_normal = read_cct("data/rna_normal.cct")
prot_tumor = read_cct("data/prot_tumor.cct")
prot_normal= read_cct("data/prot_normal.cct")

print("RNA tumor matrix:",  rna_tumor.shape)
print("RNA normal matrix:", rna_normal.shape)
print("Prot tumor matrix:", prot_tumor.shape)
print("Prot normal matrix:",prot_normal.shape)

# 3) Convert to pipeline format: rows=samples, cols=features, add label + sample_id
def to_samples_x_features(df_features_x_samples: pd.DataFrame, label: str) -> pd.DataFrame:
    out = df_features_x_samples.T.copy()
    out.index.name = "sample_id"
    out.insert(0, "label", label)
    return out.reset_index()

rna_tumor_sf  = to_samples_x_features(rna_tumor,  "tumor")
rna_normal_sf = to_samples_x_features(rna_normal, "normal")
prot_tumor_sf = to_samples_x_features(prot_tumor, "tumor")
prot_normal_sf= to_samples_x_features(prot_normal,"normal")

# 4) Combine tumor+normal for each omics
rna_all  = pd.concat([rna_tumor_sf, rna_normal_sf], ignore_index=True)
prot_all = pd.concat([prot_tumor_sf, prot_normal_sf], ignore_index=True)

# 5) Save final CSVs expected by your pipeline
rna_out  = "data/transcriptomics_processed.csv"
prot_out = "data/proteomics_processed.csv"
rna_all.to_csv(rna_out, index=False)
prot_all.to_csv(prot_out, index=False)

print("\n✅ Saved:")
print(" -", rna_out,  " shape:", rna_all.shape)
print(" -", prot_out, " shape:", prot_all.shape)

# Optional: quick sanity check
print("\nLabels (RNA):", rna_all["label"].value_counts().to_dict())
print("Labels (PROT):", prot_all["label"].value_counts().to_dict())


Downloaded: data/rna_tumor.cct
Downloaded: data/rna_normal.cct
Downloaded: data/prot_tumor.cct
Downloaded: data/prot_normal.cct
RNA tumor matrix: (18099, 110)
RNA normal matrix: (18099, 101)
Prot tumor matrix: (10316, 110)
Prot normal matrix: (10316, 101)

✅ Saved:
 - data/transcriptomics_processed.csv  shape: (211, 18101)
 - data/proteomics_processed.csv  shape: (211, 10318)

Labels (RNA): {'tumor': 110, 'normal': 101}
Labels (PROT): {'tumor': 110, 'normal': 101}


In [2]:
!pip -q install numpy pandas scikit-learn scipy

!mkdir -p src results


In [3]:
%%writefile src/io_utils.py
from __future__ import annotations
import pandas as pd

def load_omics_table(path: str, label_col: str = "label", id_col: str = "sample_id") -> pd.DataFrame:
    """
    Loads an omics table where rows are samples and columns are features.
    Must contain `label_col` with values tumor/normal (case-insensitive).
    If `id_col` exists, it is used as the index.
    """
    df = pd.read_csv(path)

    if label_col not in df.columns:
        raise ValueError(f"Missing required label column '{label_col}' in {path}")

    # Use sample_id as index if present
    if id_col in df.columns:
        df = df.set_index(id_col)

    # Clean label strings
    df[label_col] = df[label_col].astype(str).str.strip().str.lower()

    # Ensure feature columns are numeric
    feat_cols = [c for c in df.columns if c != label_col]
    df[feat_cols] = df[feat_cols].apply(pd.to_numeric, errors="coerce")

    return df

def ensure_binary_labels(labels: pd.Series) -> pd.Series:
    """
    Converts tumor/normal to 1/0.
    tumor -> 1, normal -> 0
    """
    s = labels.astype(str).str.strip().str.lower()
    valid = {"tumor", "normal"}
    uniq = set(s.unique())
    if not uniq.issubset(valid):
        raise ValueError(f"Labels must be only {valid}, got: {sorted(list(uniq))}")
    return s.map({"normal": 0, "tumor": 1}).astype(int)


Writing src/io_utils.py


In [4]:
%%writefile src/biomarker_discovery.py
from __future__ import annotations
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

def benjamini_hochberg(pvals: np.ndarray) -> np.ndarray:
    pvals = np.asarray(pvals, dtype=float)
    n = len(pvals)
    order = np.argsort(pvals)
    ranked = pvals[order]
    q = np.empty(n, dtype=float)

    prev = 1.0
    for i in range(n - 1, -1, -1):
        rank = i + 1
        val = (ranked[i] * n) / rank
        prev = min(prev, val)
        q[i] = prev

    out = np.empty(n, dtype=float)
    out[order] = np.clip(q, 0, 1)
    return out

def rank_features_ttest(X: pd.DataFrame, y: pd.Series, top_k: int = 50) -> pd.DataFrame:
    """
    Ranks features by tumor vs normal difference using:
    - Welch's t-test p-value
    - effect size (mean tumor - mean normal)
    Missing values are imputed with feature median.
    """
    if len(X) != len(y):
        raise ValueError("X and y must have same number of samples")

    Xn = X.copy()
    med = Xn.median(axis=0, skipna=True)
    Xn = Xn.fillna(med)

    tumor = Xn[y == 1]
    normal = Xn[y == 0]

    results = []
    for feat in Xn.columns:
        a = tumor[feat].values
        b = normal[feat].values
        stat, p = ttest_ind(a, b, equal_var=False)
        effect = float(np.mean(a) - np.mean(b))
        results.append((feat, effect, abs(effect), float(p)))

    df = pd.DataFrame(results, columns=["feature", "effect_tumor_minus_normal", "abs_effect", "p_value"])
    df = df.sort_values(["p_value", "abs_effect"], ascending=[True, False]).reset_index(drop=True)
    df["fdr_bh"] = benjamini_hochberg(df["p_value"].values)
    return df.head(top_k).copy()


Writing src/biomarker_discovery.py


In [5]:
%%writefile src/integration.py
from __future__ import annotations
import pandas as pd

def integrate_by_overlap(rna_rank: pd.DataFrame, prot_rank: pd.DataFrame) -> pd.DataFrame:
    """
    Integration strategy: overlap by feature name (gene/protein symbol).
    Returns merged evidence and prioritizes features supported in both omics.
    """
    rna = rna_rank.copy().rename(columns={
        "effect_tumor_minus_normal": "rna_effect",
        "p_value": "rna_p",
        "fdr_bh": "rna_fdr"
    })
    prot = prot_rank.copy().rename(columns={
        "effect_tumor_minus_normal": "prot_effect",
        "p_value": "prot_p",
        "fdr_bh": "prot_fdr"
    })

    merged = pd.merge(
        rna[["feature", "rna_effect", "rna_p", "rna_fdr"]],
        prot[["feature", "prot_effect", "prot_p", "prot_fdr"]],
        on="feature",
        how="outer"
    )

    merged["source"] = merged.apply(
        lambda r: "RNA&PROT" if pd.notna(r["rna_p"]) and pd.notna(r["prot_p"])
        else ("RNA_only" if pd.notna(r["rna_p"]) else "PROT_only"),
        axis=1
    )

    merged["combined_score"] = merged.apply(
        lambda r: min(r["rna_p"], r["prot_p"]) if r["source"] == "RNA&PROT"
        else (r["rna_p"] if r["source"] == "RNA_only" else r["prot_p"]),
        axis=1
    )

    order = {"RNA&PROT": 0, "RNA_only": 1, "PROT_only": 2}
    merged["source_rank"] = merged["source"].map(order)
    merged = merged.sort_values(["source_rank", "combined_score"], ascending=[True, True]).drop(columns=["source_rank"])
    merged = merged.reset_index(drop=True)
    return merged


Writing src/integration.py


In [6]:
%%writefile src/ml_panel.py
from __future__ import annotations
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

def _build_model() -> Pipeline:
    return Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=5000, random_state=42))
    ])

def _cv_eval(model, X, y, cv) -> dict:
    if X.shape[1] == 0:
        return {"roc_auc_mean": np.nan, "roc_auc_std": np.nan, "acc_mean": np.nan, "acc_std": np.nan}
    scoring = {"roc_auc": "roc_auc", "accuracy": "accuracy"}
    out = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=False)
    return {
        "roc_auc_mean": float(np.mean(out["test_roc_auc"])),
        "roc_auc_std": float(np.std(out["test_roc_auc"])),
        "acc_mean": float(np.mean(out["test_accuracy"])),
        "acc_std": float(np.std(out["test_accuracy"])),
    }

def evaluate_panels(X_rna: pd.DataFrame, X_prot: pd.DataFrame, y: pd.Series,
                    rna_panel: list[str], prot_panel: list[str], int_panel: list[str]) -> pd.DataFrame:
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    rna_feats = [f for f in rna_panel if f in X_rna.columns]
    prot_feats = [f for f in prot_panel if f in X_prot.columns]

    # Integrated = concatenate matched features from RNA and Proteome (prefix to avoid name collisions)
    int_rna = [f for f in int_panel if f in X_rna.columns]
    int_prot = [f for f in int_panel if f in X_prot.columns]
    X_int = pd.concat([X_rna[int_rna].add_prefix("RNA_"),
                       X_prot[int_prot].add_prefix("PROT_")], axis=1)

    results = []
    results.append({"panel": "RNA_only", "n_features": len(rna_feats), **_cv_eval(_build_model(), X_rna[rna_feats], y, cv)})
    results.append({"panel": "PROT_only", "n_features": len(prot_feats), **_cv_eval(_build_model(), X_prot[prot_feats], y, cv)})
    results.append({"panel": "Integrated", "n_features": int(X_int.shape[1]), **_cv_eval(_build_model(), X_int, y, cv)})

    return pd.DataFrame(results)


Writing src/ml_panel.py


In [7]:
%%writefile run_pipeline.py
import argparse
from pathlib import Path
import json
import pandas as pd

from src.io_utils import load_omics_table, ensure_binary_labels
from src.biomarker_discovery import rank_features_ttest
from src.integration import integrate_by_overlap
from src.ml_panel import evaluate_panels

def main():
    parser = argparse.ArgumentParser(description="Lung cancer multi-omics biomarker discovery pipeline")
    parser.add_argument("--rna", required=True, help="Path to transcriptomics_processed.csv")
    parser.add_argument("--prot", required=True, help="Path to proteomics_processed.csv")
    parser.add_argument("--out", default="results", help="Output directory")
    parser.add_argument("--top_k", type=int, default=50, help="Top-K features to consider from each omics")
    parser.add_argument("--panel_k", type=int, default=10, help="Panel size used for ML evaluation")
    parser.add_argument("--id_col", default="sample_id", help="Sample ID column name (if present)")
    args = parser.parse_args()

    out_dir = Path(args.out)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load
    rna_df = load_omics_table(args.rna, label_col="label", id_col=args.id_col)
    prot_df = load_omics_table(args.prot, label_col="label", id_col=args.id_col)

    # Align samples if sample_id overlaps; else truncate to min length
    X_rna = rna_df.drop(columns=["label"])
    X_prot = prot_df.drop(columns=["label"])

    common = X_rna.index.intersection(X_prot.index)
    if len(common) > 0:
        X_rna = X_rna.loc[common]
        X_prot = X_prot.loc[common]
        y = ensure_binary_labels(rna_df.loc[common, "label"])
    else:
        n = min(len(X_rna), len(X_prot))
        X_rna = X_rna.iloc[:n].copy()
        X_prot = X_prot.iloc[:n].copy()
        y = ensure_binary_labels(rna_df["label"].iloc[:n])

    # Rank features
    rna_rank = rank_features_ttest(X_rna, y, top_k=args.top_k)
    prot_rank = rank_features_ttest(X_prot, y, top_k=args.top_k)

    rna_rank.to_csv(out_dir / "top_transcriptomic_biomarkers.csv", index=False)
    prot_rank.to_csv(out_dir / "top_proteomic_biomarkers.csv", index=False)

    # Integrate
    integrated = integrate_by_overlap(rna_rank, prot_rank)
    integrated.to_csv(out_dir / "integrated_biomarker_panel.csv", index=False)

    # Panels
    rna_panel = rna_rank["feature"].head(args.panel_k).tolist()
    prot_panel = prot_rank["feature"].head(args.panel_k).tolist()
    int_panel = integrated.query("source == 'RNA&PROT'")["feature"].head(args.panel_k).tolist()

    # If overlap is tiny, fall back to top integrated rows
    if len(int_panel) == 0:
        int_panel = integrated["feature"].head(args.panel_k).tolist()

    perf = evaluate_panels(X_rna, X_prot, y, rna_panel, prot_panel, int_panel)
    perf.to_csv(out_dir / "model_performance.csv", index=False)

    summary = {
        "n_samples_used": int(len(y)),
        "n_rna_features": int(X_rna.shape[1]),
        "n_prot_features": int(X_prot.shape[1]),
        "top_k_ranked": int(args.top_k),
        "panel_k_used": int(args.panel_k),
        "integrated_overlap_total_in_topk": int((integrated["source"] == "RNA&PROT").sum()),
        "panels": {"rna_panel": rna_panel, "protein_panel": prot_panel, "integrated_panel": int_panel},
    }
    with open(out_dir / "run_summary.json", "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)

    print("✅ Done. Results saved to:", out_dir.resolve())
    print(perf)

if __name__ == "__main__":
    main()


Writing run_pipeline.py


In [8]:
!python run_pipeline.py \
  --rna data/transcriptomics_processed.csv \
  --prot data/proteomics_processed.csv \
  --out results \
  --top_k 50 \
  --panel_k 10


✅ Done. Results saved to: /content/results
        panel  n_features  roc_auc_mean  roc_auc_std  acc_mean   acc_std
0    RNA_only          10           1.0          0.0  1.000000  0.000000
1   PROT_only          10           1.0          0.0  1.000000  0.000000
2  Integrated          16           1.0          0.0  0.995238  0.009524


In [9]:
import pandas as pd

print("Top RNA biomarkers:")
display(pd.read_csv("results/top_transcriptomic_biomarkers.csv").head(10))

print("Top Protein biomarkers:")
display(pd.read_csv("results/top_proteomic_biomarkers.csv").head(10))

print("Integrated panel (top rows):")
display(pd.read_csv("results/integrated_biomarker_panel.csv").head(15))

print("Model performance:")
display(pd.read_csv("results/model_performance.csv"))


Top RNA biomarkers:


,feature,effect_tumor_minus_normal,abs_effect,p_value,fdr_bh
0,ACVRL1,-2.626476,2.626476,8.586725e-95,1.554111e-90
1,TEK,-3.387976,3.387976,1.792895e-92,1.622480e-88
2,PYCR1,3.709986,3.709986,9.355264e-92,5.644031e-88
3,RAMP2,-3.062312,3.062312,1.357765e-91,6.143547e-88
4,LDB2,-2.589410,2.589410,1.627054e-89,5.889609e-86
5,CDH5,-2.805698,2.805698,7.969205e-88,2.403911e-84
6,CALCRL,-2.817088,2.817088,1.148994e-87,2.970806e-84
7,CCM2L,-2.855968,2.855968,2.597818e-87,5.877238e-84
8,RTKN2,-3.956522,3.956522,3.628437e-87,7.095158e-84
9,RAMP3,-3.524913,3.524913,3.920193e-87,7.095158e-84


Top Protein biomarkers:


,feature,effect_tumor_minus_normal,abs_effect,p_value,fdr_bh
0,CAVIN2,-7.170160,7.170160,2.848387e-98,2.938396e-94
1,EHD2,-6.654693,6.654693,2.568121e-97,1.324637e-93
2,PALM2-AKAP2,-4.839326,4.839326,6.620017e-93,1.818201e-89
3,CAVIN1,-6.522512,6.522512,7.050023e-93,1.818201e-89
4,SHANK3,-4.448180,4.448180,2.997661e-92,6.184775e-89
5,CLIC5,-7.245507,7.245507,2.304452e-90,3.962121e-87
6,STARD13,-2.963181,2.963181,2.134214e-88,3.145221e-85
7,HSPA12B,-5.957528,5.957528,2.640745e-88,3.405240e-85
8,KANK3,-5.866009,5.866009,3.910302e-88,4.482074e-85
9,RASIP1,-4.977640,4.977640,2.936654e-87,3.029453e-84


Integrated panel (top rows):


,feature,rna_effect,rna_p,rna_fdr,prot_effect,prot_p,prot_fdr,source,combined_score
0,STARD13,-1.983819,1.842771e-82,1.667615e-79,-2.963181,2.134214e-88,3.145221e-85,RNA&PROT,2.134214e-88
1,HSPA12B,-2.779830,4.687831e-86,7.070422e-83,-5.957528,2.640745e-88,3.405240e-85,RNA&PROT,2.640745e-88
2,KANK3,-3.015885,4.512273e-80,2.722254e-77,-5.866009,3.910302e-88,4.482074e-85,RNA&PROT,3.910302e-88
3,CDH5,-2.805698,7.969205e-88,2.403911e-84,-4.918170,3.024149e-82,1.418051e-79,RNA&PROT,7.969205e-88
4,RTKN2,-3.956522,3.628437e-87,7.095158e-84,-6.908995,7.923612e-86,6.811665e-83,RNA&PROT,3.628437e-87
5,JAM2,-2.563365,5.581187e-85,7.770301e-82,-4.966841,4.641733e-78,9.576823e-76,RNA&PROT,5.581187e-85
6,MMRN2,-2.207862,1.596944e-76,7.049533e-74,-4.874657,3.518271e-81,1.166129e-78,RNA&PROT,3.518271e-81
7,WWC2,-1.869552,9.441125e-75,3.559894e-72,-3.183496,4.422646e-78,9.311025e-76,RNA&PROT,4.422646e-78
8,ACVRL1,-2.626476,8.586725e-95,1.554111e-90,NaN,NaN,NaN,RNA_only,8.586725e-95
9,TEK,-3.387976,1.792895e-92,1.622480e-88,NaN,NaN,NaN,RNA_only,1.792895e-92


Model performance:


,panel,n_features,roc_auc_mean,roc_auc_std,acc_mean,acc_std
0,RNA_only,10,1.0,0.0,1.000000,0.000000
1,PROT_only,10,1.0,0.0,1.000000,0.000000
2,Integrated,16,1.0,0.0,0.995238,0.009524
